# loan_predictionMishaely

In [419]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


import all dependency for loan prediction by  neural network using tensor flow framework in and ignore warning

In [420]:
!pip install catboost

In [421]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import roc_auc_score
import lightgbm as lgb
import catboost as cb
import warnings
warnings.filterwarnings('ignore')

Loading datasets three from train side and three from test side

In [422]:
#path for each data
train1='/content/drive/MyDrive/loan_prediction/traindemographics.csv'
train2='/content/drive/MyDrive/loan_prediction/trainperf.csv'
train3='/content/drive/MyDrive/loan_prediction/trainprevloans.csv'
test1='/content/drive/MyDrive/loan_prediction/testdemographics.csv'
test2='/content/drive/MyDrive/loan_prediction/testperf.csv'
test3='/content/drive/MyDrive/loan_prediction/testprevloans.csv'
SampleSubmision='/content/drive/MyDrive/loan_prediction/SampleSubmission.csv'

In [423]:
#load data from path above
train_demographics=pd.read_csv(train1)
train_perf=pd.read_csv(train2)
train_prevloans=pd.read_csv(train3)
test_demographics=pd.read_csv(test1)
test_perf=pd.read_csv(test2)
test_prevloans=pd.read_csv(test3)
SampleSubmission=pd.read_csv(SampleSubmision)

Dispaly data

In [424]:
train_demographics.head(2)

,customerid,birthdate,bank_account_type,longitude_gps,latitude_gps,bank_name_clients,bank_branch_clients,employment_status_clients,level_of_education_clients
0,8a858e135cb22031015cbafc76964ebd,1973-10-10 00:00:00.000000,Savings,3.319219,6.528604,GT Bank,NaN,NaN,NaN
1,8a858e275c7ea5ec015c82482d7c3996,1986-01-21 00:00:00.000000,Savings,3.325598,7.119403,Sterling Bank,NaN,Permanent,NaN


In [425]:
test_demographics.head(2)

,customerid,birthdate,bank_account_type,longitude_gps,latitude_gps,bank_name_clients,bank_branch_clients,employment_status_clients,level_of_education_clients
0,8a858f305c8dd672015c93b1db645db4,1976-08-28 00:00:00.000000,Savings,5.296628,7.593965,Heritage Bank,NaN,Permanent,NaN
1,8a858f085a477386015a47fb049e49ca,1978-06-23 00:00:00.000000,Savings,3.294513,6.596602,UBA,NaN,Permanent,NaN


In [426]:
sample=SampleSubmission.head(2)
display(sample)

,customerid,Good_Bad_flag
0,8a28afc7474813a40147639ec637156b,1
1,8a3735d5518aba7301518ac34413010d,1


Data preproccessing and handling missing values

In [427]:
train_demographics.drop(columns=['bank_branch_clients','level_of_education_clients'], axis=1, inplace=True)
test_demographics.drop(columns=['bank_branch_clients','level_of_education_clients'], axis=1, inplace=True)


In [428]:
train_perf.drop(columns=['systemloanid','loannumber','referredby',], axis=1, inplace=True)
test_perf.drop(columns=['systemloanid','loannumber','referredby',], axis=1, inplace=True )

In [429]:
train_prevloans.drop(columns=['referredby','loannumber','systemloanid'], axis=1, inplace=True)
test_prevloans.drop(columns=['referredby','systemloanid','loannumber'], axis=1, inplace=True)

In [430]:
from datetime import datetime

train_demographics['birthdate'] = pd.to_datetime(train_demographics['birthdate'], errors='coerce')
current_date = pd.to_datetime(datetime.today().date())
train_demographics['age'] = (current_date - train_demographics['birthdate']).dt.days // 365

#test side
test_demographics['birthdate'] = pd.to_datetime(test_demographics['birthdate'], errors='coerce')
test_demographics['age'] = (current_date - test_demographics['birthdate']).dt.days // 365

#Drop useless
train_demographics.drop(columns=['birthdate'], axis=1, inplace=True)
test_demographics.drop(columns=['birthdate'], axis=1, inplace=True)

# Frequency encoding for bank_name_clients
bank_name_freq = train_demographics['bank_name_clients'].value_counts(normalize=True)

train_demographics['bank_name_clients'] = train_demographics['bank_name_clients'].map(bank_name_freq)
test_demographics['bank_name_clients'] = test_demographics['bank_name_clients'].map(bank_name_freq).fillna(0)

# One-hot encode bank_account_type
train_demographics['bank_account_type'] = train_demographics['bank_account_type'].apply(lambda x: 1 if x == 'Savings' else 0)
test_demographics['bank_account_type'] = test_demographics['bank_account_type'].apply(lambda x: 1 if x == 'Savings' else 0)


# Order encode employment_status_clients
# Fill missing values with a placeholder
train_demographics['employment_status_clients'].fillna('Unknown', inplace=True)
test_demographics['employment_status_clients'].fillna('Unknown', inplace=True)

# Get all unique values from both train and test data for fitting the encoder
all_employment_statuses = pd.concat([train_demographics['employment_status_clients'], test_demographics['employment_status_clients']]).unique()

encoder = LabelEncoder()
encoder.fit(all_employment_statuses) # Fit on all unique values

train_demographics['employment_status_clients'] = encoder.transform(train_demographics['employment_status_clients'])
test_demographics['employment_status_clients'] = encoder.transform(test_demographics['employment_status_clients'])

train_demographics.head(3)

,customerid,bank_account_type,longitude_gps,latitude_gps,bank_name_clients,employment_status_clients,age
0,8a858e135cb22031015cbafc76964ebd,1,3.319219,6.528604,0.367694,6,51
1,8a858e275c7ea5ec015c82482d7c3996,1,3.325598,7.119403,0.012655,1,39
2,8a858e5b5bd99460015bdc95cd485634,1,5.746100,5.563174,0.023700,6,38


# let jump to performance datasets



In [431]:
#come to performance datasets
from datetime import datetime

# Convert dates to datetime format
train_perf['creationdate'] = pd.to_datetime(train_perf['creationdate'], errors='coerce')
train_perf['approveddate'] = pd.to_datetime(train_perf['approveddate'], errors='coerce')

# Create new features
train_perf['processing_hours'] = (train_perf['approveddate'] - train_perf['creationdate']).dt.total_seconds() / 3600
# Financial ratios
train_perf['interest_ratio'] = (train_perf['totaldue'] - train_perf['loanamount']) / train_perf['loanamount']
train_perf['daily_due'] = train_perf['totaldue'] / train_perf['termdays']

# Log transformations to reduce skewness
train_perf['log_loanamount'] = np.log1p(train_perf['loanamount'])
train_perf['log_totaldue'] = np.log1p(train_perf['totaldue'])

# Drop original date columns if not needed
train_perf.drop(columns=['creationdate', 'approveddate'], inplace=True)


# Convert dates to datetime format
test_perf['creationdate'] = pd.to_datetime(test_perf['creationdate'], errors='coerce')
test_perf['approveddate'] = pd.to_datetime(test_perf['approveddate'], errors='coerce')

# Create new features
test_perf['processing_hours'] = (test_perf['approveddate'] - test_perf['creationdate']).dt.total_seconds() / 3600

# Financial ratios
test_perf['interest_ratio'] = (test_perf['totaldue'] - test_perf['loanamount']) / test_perf['loanamount']
test_perf['daily_due'] = test_perf['totaldue'] / test_perf['termdays']

# Log transformations to reduce skewness
test_perf['log_loanamount'] = np.log1p(test_perf['loanamount'])
test_perf['log_totaldue'] = np.log1p(test_perf['totaldue'])

# Impute missing values in processing_hours with the mean
train_perf['processing_hours'].fillna(train_perf['processing_hours'].mean(), inplace=True)
test_perf['processing_hours'].fillna(test_perf['processing_hours'].mean(), inplace=True)

# Encode the target variable: 'Good' as 1, 'Bad' as 0
encoder = LabelEncoder()
train_perf['good_bad_flag'] = encoder.fit_transform(train_perf['good_bad_flag'])

# Verify the mapping
mapping = dict(zip(encoder.classes_, encoder.transform(encoder.classes_)))
print(f"Encoding mapping: {mapping}")


# Drop original date columns if not needed
test_perf.drop(columns=['creationdate', 'approveddate'], inplace=True)



Encoding mapping: {'Bad': np.int64(0), 'Good': np.int64(1)}


In [432]:
train_perf.head(2)

,customerid,loanamount,totaldue,termdays,good_bad_flag,processing_hours,interest_ratio,daily_due,log_loanamount,log_totaldue
0,8a2a81a74ce8c05d014cfb32a0da1049,30000.0,34500.0,30,1,1.002500,0.15,1150.0,10.308986,10.448744
1,8a85886e54beabf90154c0a29ae757c0,15000.0,17250.0,30,1,1.006389,0.15,575.0,9.615872,9.755625


# let jump to prevloans datasets

In [433]:
# Convert date columns to datetime
date_cols = ['approveddate', 'creationdate', 'closeddate', 'firstduedate', 'firstrepaiddate']
for col in date_cols:
    train_prevloans[col] = pd.to_datetime(train_prevloans[col], errors='coerce')

# --- Temporal Features ---

train_prevloans['processing_hours'] = (train_prevloans['approveddate'] - train_prevloans['creationdate']).dt.total_seconds() / 3600
train_prevloans['loan_duration_actual'] = (train_prevloans['closeddate'] - train_prevloans['creationdate']).dt.days
train_prevloans['time_to_first_repay'] = (train_prevloans['firstrepaiddate'] - train_prevloans['firstduedate']).dt.days
train_prevloans['days_overdue'] = (train_prevloans['closeddate'] - train_prevloans['firstduedate']).dt.days

train_prevloans['loan_age'] = (pd.Timestamp.now() - train_prevloans['creationdate']).dt.days

# --- Financial Features ---
train_prevloans['repayment_ratio'] = train_prevloans['totaldue'] / train_prevloans['loanamount']
train_prevloans['daily_repayment_rate'] = train_prevloans['totaldue'] / train_prevloans['termdays'].replace(0, np.nan)
train_prevloans['interest_amount'] = train_prevloans['totaldue'] - train_prevloans['loanamount']
train_prevloans['loan_size_category'] = pd.qcut(train_prevloans['loanamount'], q=3, labels=['small', 'medium', 'large'])

# --- Behavioral Features ---
train_prevloans['early_repayment'] = (train_prevloans['firstrepaiddate'] < train_prevloans['firstduedate']).astype(int)
train_prevloans['missed_first_payment'] = (train_prevloans['firstrepaiddate'] > train_prevloans['firstduedate']).astype(int)
train_prevloans['closure_speed'] = (train_prevloans['closeddate'] - train_prevloans['creationdate']).dt.days

# --- Interaction Features ---
train_prevloans['loanamount_x_termdays'] = train_prevloans['loanamount'] * train_prevloans['termdays']
train_prevloans['repayment_ratio_x_loan_age'] = train_prevloans['repayment_ratio'] * train_prevloans['loan_age']

# --- Handle Missing Values ---
train_prevloans.fillna({
    'processing_days': -1,
    'loan_duration_actual': -1,
    'time_to_first_repay': -1,
    'days_overdue': -1,
    'repayment_ratio': 0,
    'daily_repayment_rate': 0,
    'interest_amount': 0
}, inplace=True)

train_prevloans.drop(columns=['approveddate', 'creationdate', 'closeddate', 'firstduedate', 'firstrepaiddate'], inplace=True)

# Define the order of categories
category_order = ['small', 'medium', 'large']

# Map categories to numerical values using the defined order
train_prevloans['loan_size'] = train_prevloans['loan_size_category'].apply(lambda x: category_order.index(x) if pd.notnull(x) else -1)

# Drop the original 'loan_size_category' column
train_prevloans.drop(columns=['loan_size_category'], inplace=True)


In [434]:
# Convert date columns to datetime
date_cols = ['approveddate', 'creationdate', 'closeddate', 'firstduedate', 'firstrepaiddate']
for col in date_cols:
    test_prevloans[col] = pd.to_datetime(test_prevloans[col], errors='coerce')

# --- Temporal Features ---
test_prevloans['processing_hours'] = (test_prevloans['approveddate'] - test_prevloans['creationdate']).dt.total_seconds() / 3600
test_prevloans['loan_duration_actual'] = (test_prevloans['closeddate'] - test_prevloans['creationdate']).dt.days
test_prevloans['time_to_first_repay'] = (test_prevloans['firstrepaiddate'] - test_prevloans['firstduedate']).dt.days
test_prevloans['days_overdue'] = (test_prevloans['closeddate'] - test_prevloans['firstduedate']).dt.days
test_prevloans['loan_age'] = (pd.Timestamp.now() - test_prevloans['creationdate']).dt.days

# --- Financial Features ---
test_prevloans['repayment_ratio'] = test_prevloans['totaldue'] / test_prevloans['loanamount']
test_prevloans['daily_repayment_rate'] = test_prevloans['totaldue'] / test_prevloans['termdays'].replace(0, np.nan)
test_prevloans['interest_amount'] = test_prevloans['totaldue'] - test_prevloans['loanamount']
test_prevloans['loan_size_category'] = pd.qcut(test_prevloans['loanamount'], q=3, labels=['small', 'medium', 'large'])

# --- Behavioral Features ---
test_prevloans['early_repayment'] = (test_prevloans['firstrepaiddate'] < test_prevloans['firstduedate']).astype(int)
test_prevloans['missed_first_payment'] = (test_prevloans['firstrepaiddate'] > test_prevloans['firstduedate']).astype(int)
test_prevloans['closure_speed'] = (test_prevloans['closeddate'] - test_prevloans['creationdate']).dt.days

# --- Interaction Features ---
test_prevloans['loanamount_x_termdays'] = test_prevloans['loanamount'] * test_prevloans['termdays']
test_prevloans['repayment_ratio_x_loan_age'] = test_prevloans['repayment_ratio'] * test_prevloans['loan_age']

# --- Handle Missing Values ---
test_prevloans.fillna({
    'processing_days': -1,
    'loan_duration_actual': -1,
    'time_to_first_repay': -1,
    'days_overdue': -1,
    'repayment_ratio': 0,
    'daily_repayment_rate': 0,
    'interest_amount': 0
}, inplace=True)

test_prevloans.drop(columns=['approveddate', 'creationdate', 'closeddate', 'firstduedate', 'firstrepaiddate'], inplace=True)

# Define the order of categories
category_order = ['small', 'medium', 'large']

# Map categories to numerical values using the defined order
test_prevloans['loan_size_category_encoded'] = test_prevloans['loan_size_category'].apply(lambda x: category_order.index(x) if pd.notnull(x) else -1)

# Drop the original 'loan_size_category' column
test_prevloans.drop(columns=['loan_size_category'], inplace=True)


# aggregate the prevloans datasets

In [435]:
# Aggregate prevloans datasets
customer_summary = train_prevloans.groupby('customerid').agg(
             loannumber_count = ('loanamount','count'), # Using loanamount to count loans per customer
             loannumber_sum = ('loanamount','sum'), # Sum of loan amounts


             closed_creation_diff_sum = ("closure_speed", "sum"), # Renamed to closure_speed


             repaid_due_diff_sum = ("time_to_first_repay", "sum"), # Renamed to time_to_first_repay


             creation_approved_diff_sum = ("processing_hours", "sum"), # Renamed to processing_hours


             loanamount_sum = ("loanamount", "sum"),

             totaldue_sum = ("totaldue", "sum"),


             termdays_sum = ("termdays", "sum"),


             totaldue_loanamount_diff_sum = ("interest_amount", "sum"), # Renamed to interest_amount
).reset_index()

display(customer_summary.head())

,customerid,loannumber_count,loannumber_sum,closed_creation_diff_sum,repaid_due_diff_sum,creation_approved_diff_sum,loanamount_sum,totaldue_sum,termdays_sum,totaldue_loanamount_diff_sum
0,8a1088a0484472eb01484669e3ce4e0b,1,10000.0,22,6,1.018889,10000.0,11500.0,15,1500.0
1,8a1a1e7e4f707f8b014f797718316cad,4,70000.0,125,-1,4.025278,70000.0,89500.0,150,19500.0
2,8a1a32fc49b632520149c3b8fdf85139,7,90000.0,129,-3,7.036111,90000.0,106500.0,135,16500.0
3,8a1eb5ba49a682300149c3c068b806c7,8,130000.0,228,-25,8.036667,130000.0,162400.0,270,32400.0
4,8a1edbf14734127f0147356fdb1b1eb2,2,20000.0,36,-8,2.020833,20000.0,24500.0,45,4500.0


In [436]:
# Aggregate prevloans datasets
customer_summary02 = test_prevloans.groupby('customerid').agg(
             loannumber_count = ('loanamount','count'), # Using loanamount to count loans per customer
             loannumber_sum = ('loanamount','sum'), # Sum of loan amounts


             closed_creation_diff_sum = ("closure_speed", "sum"), # Renamed to closure_speed


             repaid_due_diff_sum = ("time_to_first_repay", "sum"), # Renamed to time_to_first_repay


             creation_approved_diff_sum = ("processing_hours", "sum"), # Renamed to processing_hours


             loanamount_sum = ("loanamount", "sum"),

             totaldue_sum = ("totaldue", "sum"),


             termdays_sum = ("termdays", "sum"),


             totaldue_loanamount_diff_sum = ("interest_amount", "sum"), # Renamed to interest_amount
).reset_index()

display(customer_summary02.head())

,customerid,loannumber_count,loannumber_sum,closed_creation_diff_sum,repaid_due_diff_sum,creation_approved_diff_sum,loanamount_sum,totaldue_sum,termdays_sum,totaldue_loanamount_diff_sum
0,8a28afc7474813a40147639ec637156b,10,178000.0,255,-33,10.036111,178000.0,210900.0,315,32900.0
1,8a3735d5518aba7301518ac34413010d,6,70000.0,86,-76,6.033333,70000.0,88000.0,165,18000.0
2,8a76e7d443e6e97c0143ed099d102b1d,2,20000.0,27,-4,2.009167,20000.0,23000.0,30,3000.0
3,8a818823525dceef01525deda2480384,9,170000.0,354,1,24.020833,170000.0,209975.0,360,39975.0
4,8a818926522ea5ef01523aff15c37482,2,20000.0,81,17,2.017500,20000.0,26000.0,60,6000.0


# mergig datasets to create train and test datasets only

In [437]:
train=train_perf.merge(train_demographics, on='customerid', how='left')
train=train.merge(customer_summary, on='customerid', how='left')

In [438]:
test=test_perf.merge(test_demographics, on='customerid', how='left')
test=test.merge(customer_summary02, on='customerid', how='left')

In [439]:
train.isnull().sum() , test.isnull().sum()

(customerid                         0
 loanamount                         0
 totaldue                           0
 termdays                           0
 good_bad_flag                      0
 processing_hours                   0
 interest_ratio                     0
 daily_due                          0
 log_loanamount                     0
 log_totaldue                       0
 bank_account_type               1099
 longitude_gps                   1099
 latitude_gps                    1099
 bank_name_clients               1099
 employment_status_clients       1099
 age                             1099
 loannumber_count                   9
 loannumber_sum                     9
 closed_creation_diff_sum           9
 repaid_due_diff_sum                9
 creation_approved_diff_sum         9
 loanamount_sum                     9
 totaldue_sum                       9
 termdays_sum                       9
 totaldue_loanamount_diff_sum       9
 dtype: int64,
 customerid                        

In [440]:
# Impute missing values in train and test DataFrames

# For numerical columns with missing values, use mean or median
numerical_cols_to_impute = ['loanamount', 'totaldue', 'termdays', 'processing_hours', 'interest_ratio', 'daily_due',
                           'log_loanamount', 'log_totaldue', 'loannumber_count', 'loannumber_sum',
                           'closed_creation_diff_sum', 'repaid_due_diff_sum', 'creation_approved_diff_sum',
                           'loanamount_sum', 'totaldue_sum', 'termdays_sum', 'totaldue_loanamount_diff_sum']

for col in numerical_cols_to_impute:
    if col in train.columns:
        train[col].fillna(train[col].median(), inplace=True) # Using median for robustness to outliers
    if col in test.columns:
        test[col].fillna(test[col].median(), inplace=True) # Using median for robustness to outliers

# good_bad_flag is the target variable in train, drop rows with missing values
if 'good_bad_flag' in train.columns:
    train.dropna(subset=['good_bad_flag'], inplace=True)

# Verify that missing values are handled
print("Missing values after imputation in train:")
print(train.isnull().sum())
print("\nMissing values after imputation in test:")
print(test.isnull().sum())

Missing values after imputation in train:
customerid                         0
loanamount                         0
totaldue                           0
termdays                           0
good_bad_flag                      0
processing_hours                   0
interest_ratio                     0
daily_due                          0
log_loanamount                     0
log_totaldue                       0
bank_account_type               1099
longitude_gps                   1099
latitude_gps                    1099
bank_name_clients               1099
employment_status_clients       1099
age                             1099
loannumber_count                   0
loannumber_sum                     0
closed_creation_diff_sum           0
repaid_due_diff_sum                0
creation_approved_diff_sum         0
loanamount_sum                     0
totaldue_sum                       0
termdays_sum                       0
totaldue_loanamount_diff_sum       0
dtype: int64

Missing values afte

# Train the models using catboost and lgb algorithsm

In [441]:
test.shape

(1450, 24)

In [442]:
SampleSubmission.shape

(1450, 2)

In [443]:
#drop unimportant columns
train.drop(columns=['customerid'], axis=1, inplace=True)
test.drop(columns=['customerid'], axis=1, inplace=True)

In [444]:
train.columns

Index(['loanamount', 'totaldue', 'termdays', 'good_bad_flag',
       'processing_hours', 'interest_ratio', 'daily_due', 'log_loanamount',
       'log_totaldue', 'bank_account_type', 'longitude_gps', 'latitude_gps',
       'bank_name_clients', 'employment_status_clients', 'age',
       'loannumber_count', 'loannumber_sum', 'closed_creation_diff_sum',
       'repaid_due_diff_sum', 'creation_approved_diff_sum', 'loanamount_sum',
       'totaldue_sum', 'termdays_sum', 'totaldue_loanamount_diff_sum'],
      dtype='object')

In [445]:
test.columns

Index(['loanamount', 'totaldue', 'termdays', 'processing_hours',
       'interest_ratio', 'daily_due', 'log_loanamount', 'log_totaldue',
       'bank_account_type', 'longitude_gps', 'latitude_gps',
       'bank_name_clients', 'employment_status_clients', 'age',
       'loannumber_count', 'loannumber_sum', 'closed_creation_diff_sum',
       'repaid_due_diff_sum', 'creation_approved_diff_sum', 'loanamount_sum',
       'totaldue_sum', 'termdays_sum', 'totaldue_loanamount_diff_sum'],
      dtype='object')

In [446]:
# Split the training data into features (X) and target (y)
X = train.drop(columns=['good_bad_flag'])
y = train['good_bad_flag']

# Split data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Initialize and train the LightGBM model
lgb_model = lgb.LGBMClassifier(random_state=42)
lgb_model.fit(X_train, y_train)

# Predict probabilities on the validation set
lgb_pred_proba = lgb_model.predict_proba(X_val)[:, 1]

# Evaluate LightGBM model
lgb_roc_auc = roc_auc_score(y_val, lgb_pred_proba)
print(f"LightGBM ROC AUC: {lgb_roc_auc}")

# Initialize and train the CatBoost model
cat_model = cb.CatBoostClassifier(verbose=0, random_state=42) # Set verbose=0 to reduce output
cat_model.fit(X_train, y_train)

# Predict probabilities on the validation set
cat_pred_proba = cat_model.predict_proba(X_val)[:, 1]

# Evaluate CatBoost model
cat_roc_auc = roc_auc_score(y_val, cat_pred_proba)
print(f"CatBoost ROC AUC: {cat_roc_auc}")

[LightGBM] [Info] Number of positive: 2738, number of negative: 762
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000331 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2194
[LightGBM] [Info] Number of data points in the train set: 3500, number of used features: 23
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.782286 -> initscore=1.279036
[LightGBM] [Info] Start training from score 1.279036
LightGBM ROC AUC: 0.6710054648985363
CatBoost ROC AUC: 0.6878358237474682


In [447]:
# Predict probabilities on the test set using the CatBoost model
test_pred_proba = cat_model.predict_proba(test)[:, 1]

In [448]:
submission = pd.DataFrame({
    'customerid': SampleSubmission['customerid'],
    'Good_Bad_flag': test_pred_proba
})
display(submission.head())

,customerid,Good_Bad_flag
0,8a28afc7474813a40147639ec637156b,0.799224
1,8a3735d5518aba7301518ac34413010d,0.970484
2,8a76e7d443e6e97c0143ed099d102b1d,0.869121
3,8a818823525dceef01525deda2480384,0.916781
4,8a818926522ea5ef01523aff15c37482,0.869260


In [449]:
# Convert probabilities to binary predictions using a threshold of 0.5
submission['Good_Bad_flag'] = (submission['Good_Bad_flag'] > 0.5).astype(int)
display(submission.head())

,customerid,Good_Bad_flag
0,8a28afc7474813a40147639ec637156b,1
1,8a3735d5518aba7301518ac34413010d,1
2,8a76e7d443e6e97c0143ed099d102b1d,1
3,8a818823525dceef01525deda2480384,1
4,8a818926522ea5ef01523aff15c37482,1


from matplotlib import pyplot as plt
import seaborn as sns
_df_1.groupby('customerid').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  palette = list(sns.palettes.mpl_palette('Dark2'))
  counted = (series['Good_Bad_flag']
                .value_counts()
              .reset_index(name='counts')
              .rename({'index': 'Good_Bad_flag'}, axis=1)
              .sort_values('Good_Bad_flag', ascending=True))
  xs = counted['Good_Bad_flag']
  ys = counted['counts']
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = _df_2.sort_values('Good_Bad_flag', ascending=True)
for i, (series_name, series) in enumerate(df_sorted.groupby('customerid')):
  _plot_series(series, series_name, i)
  fig.legend(title='customerid', bbox_to_anchor=(1, 1), loc='upper left')
sns.despine(fig=fig, ax=ax)
plt.xlabel('Good_Bad_flag')
_ = plt.ylabel('count()')

In [450]:
# Save the submission file
submission.to_csv('submission.csv', index=False)